In [24]:
import hyperspy.api as hs
from matplotlib import pyplot as plt
import numpy as np
import pycrystem as pc
import itertools
plt.rcParams['image.cmap'] = 'gray'
%matplotlib tk

In [34]:
dp = pc.load("/home/pc494/Documents/data/GaN_intro/GaN_indent.blo")

In [35]:
dp = dp.inav[0:25,475:500]
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]));
peaks = dp.find_peaks('zaefferer')
#This hangs if you run it in a series of cells sometimes?!

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [36]:
if dp.axes_manager.signal_shape[0] == dp.axes_manager.signal_shape[1]:
    peaks.data = peaks.data + (dp.axes_manager.signal_shape[0])/2 * 0.0076034063260340635
else:
    raise NotImplementedError('Needs a square signal')

In [37]:
def find_max_length_peaks(peaks):
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    length_of_longest_peaks_list = 0
    for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                if peaks.data[x,y].shape[0] > length_of_longest_peaks_list:
                    length_of_longest_peaks_list = peaks.data[x,y].shape[0]
    return length_of_longest_peaks_list      

In [29]:
## This is slow and needs to be vectorised

def generate_markers_from_peaks(peaks):
    markx,marky = [],[]
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    max_peak_count = find_max_length_peaks(peaks)
    for k in np.arange(0,max_peak_count):
        mx,my = [],[]
        for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                try:
                    mx.append(peaks.inav[x,y].data[k][1])
                    my.append(peaks.inav[x,y].data[k][0])
                except IndexError: #we have fewer than 'max_peak_count' peaks, so no marker
                    mx.append(np.nan)
                    my.append(np.nan)
        markx.append(np.asarray(mx).reshape(x_size,y_size).T)
        marky.append(np.asarray(my).reshape(x_size,y_size).T)
    
    return markx,marky 

markx,marky = generate_markers_from_peaks(peaks)

Base Time is 15 seconds

In [40]:
markxT = [None] * find_max_length_peaks(peaks)
markyT = [None] * find_max_length_peaks(peaks)

a = np.array(list(itertools.zip_longest(*np.concatenate(peaks.data),fillvalue=[np.nan,np.nan])))
a = a.reshape(len(markxT),25,25,2) #getting the 2D nav back
xy = np.transpose(a,[3,0,1,2]) #switching for selection
x = xy[1]
y = xy[0]
for cnt in np.arange(len(markxT)):
    markxT[cnt] = x[cnt]
    markyT[cnt] = y[cnt]

In [41]:
dp.plot() #the requirement to include this is a bug.

#for mx,my in zip(markx,marky):
#    m = hs.markers.point(x=mx,y=my,color='gold',marker='o')
#    dp.add_marker(m,plot_marker=True,permanent=False)
for mx,my in zip(markxT,markyT):
    m = hs.markers.point(x=mx,y=my,color='gold',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=False)

In [ ]:
## Profile this,
## Then figure out how to get it into pycrystem
## Then do that, and tidy up the other stuff you've done en-route

In [ ]:
import itertools

ll = [[1, 2, 3], [1, 2]]
np.array(list(itertools.zip_longest(*ll,fillvalue=np.nan))).T

In [ ]:
alpha = np.arange(1,5).reshape(2,2)
beta = np.concatenate(alpha).reshape(2,2)
alpha == beta